In [83]:
# reqeusts, bs4 import
import requests, bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup
import json
import time

In [84]:
req_header = { "user-agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'}

In [85]:
def select_occ1_id(soup):
    occ1_id_dict = {}
    for button in soup.select("ul#occ1_div li button[data-item-val]"):
        occ1_id = button.get("data-item-val")
        occ1_name = button.find_next("span").text if button.find_next("span") else ""
        if occ1_id and occ1_name:
            occ1_id_dict[occ1_id] = occ1_name
    return occ1_id_dict


def select_occ2_id(soup, id_list):
    occ2_id_dict = {}
    # 모든 occ2 관련 버튼을 한 번에 선택하고 필터링
    all_occ2_buttons = soup.select("ul[id^='occ2_ul_'] li button[data-item-val]")
    
    for button in all_occ2_buttons:
        data_val = button.get("data-item-val")
        if data_val and "_" in data_val:
            occ1_id, occ2_id = data_val.split("_")
            if occ1_id in id_list:
                occ2_name = button.find_next("span").text if button.find_next("span") else ""
                occ2_id_dict[occ2_id] = (occ1_id, occ2_name)
    
    return occ2_id_dict

In [86]:
import time

def calculate_page_count(count_str):
    count = int(count_str.replace(",", ""))
    page = ((count - 1) // 30) + 1
    return page

def get_page_url_list(occ2_id_dict):
    count_url = "https://job.incruit.com/s_common/searchjob/v3/searchjob_getcount_ajax.asp?occ2="
    base_url = "https://job.incruit.com/jobdb_list/searchjob.asp?articlecount=30&occ2="
    
    page_url_list = []
    with requests.Session() as session:
        session.headers.update(req_header)
        
        for occ2_id in occ2_id_dict.keys():
            try:
                res = session.get(count_url + occ2_id)
                if res.ok:
                    page_num = calculate_page_count(res.text)
                    # 리스트 컴프리헨션 사용으로 루프 최적화
                    new_urls = [(occ2_id, f"{base_url}{occ2_id}&page={page_index}") 
                               for page_index in range(1, page_num + 1)]
                    page_url_list.extend(new_urls)
                else:
                    print(f"에러 코드 = {res.status_code}, occ2_id: {occ2_id}")
            except Exception as e:
                print(f"예외 발생: {e}, occ2_id: {occ2_id}")
    
    return page_url_list

def first_cell_parser(li_tag):
    first_cell = li_tag.find('div', class_='cell_first') # 기업 이름과 태그
    company = first_cell.find('a')
    return company.text

def mid_cell_parser(li_tag):
    mid_cell = li_tag.find('div', class_='cell_mid') # 채용 공고
    title = mid_cell.find('a')
    href = title["href"]
    return title.text, href

def get_employ_info(id, link, session=None):
    if session is None:
        session = requests

    employ_info = {}
    try:
        res = session.get(link, headers=req_header)
        if res.ok:
            soup = BeautifulSoup(res.text, "html.parser")
            
            info_items = soup.select('ul.jc_list li div.txt em')
            
            keys = ['고용형태', '경력', '근무지역', '학력', '급여조건']
            employ_info = {keys[i]: info_items[i].text for i in range(min(len(keys), len(info_items)))}
        else:
            print(f"에러 코드 = {res.status_code}, URL: {link}")
    except Exception as e:
        print(f"정보 추출 중 오류: {e}, URL: {link}")
    
    return employ_info

In [87]:
def classify_jobs(page_url_list, occ1_id_dict, occ2_id_dict):
    job_data = {occ1_name: {} for occ1_id, occ1_name in occ1_id_dict.items()}
    
    # HTTP 세션 재사용으로 연결 오버헤드 감소
    with requests.Session() as session:
        session.headers.update(req_header)
        
        for id2, page_url in page_url_list:
            try:
                occ1_id, occ2_name = occ2_id_dict.get(id2, (None, "알 수 없는 직업"))
                if occ1_id is None:
                    continue
                    
                occ1_name = occ1_id_dict.get(occ1_id, "알 수 없는 대분류")
                
                # 해당 소분류 초기화 (필요시)
                if occ2_name not in job_data[occ1_name]:
                    job_data[occ1_name][occ2_name] = []
                
                res = session.get(page_url)
                if res.ok:
                    soup = BeautifulSoup(res.text, "html.parser")
                    
                    # 한 번에 모든 채용 정보 추출
                    li_tag_list = soup.select("li.c_col")
                    
                    for li_tag in li_tag_list:
                        # 정보 추출 최적화
                        first_cell = li_tag.find('div', class_='cell_first')
                        company_name = first_cell.find('a').text if first_cell and first_cell.find('a') else ""
                        
                        mid_cell = li_tag.find('div', class_='cell_mid')
                        title_tag = mid_cell.find('a') if mid_cell else None
                        
                        if title_tag and title_tag.get("href"):
                            title = title_tag.text
                            title_url = title_tag["href"]
                            
                            # 동일 세션 재사용
                            employ_info = get_employ_info(id2, title_url, session)
                            
                            job_info = {
                                "회사명": company_name,
                                "채용공고": title,
                                "URL": title_url,
                                **employ_info
                            }
                            
                            job_data[occ1_name][occ2_name].append(job_info)
                        
                    # 너무 짧은 sleep은 IP 차단 위험이 있으므로 주의
                    time.sleep(0.1)  # 0.2초에서 0.1초로 단축
                else:
                    print(f"페이지 접근 오류: {page_url}, 에러 코드 = {res.status_code}")
            except Exception as e:
                print(f"오류 발생: {e}, occ2_id: {id2}, URL: {page_url}")
    
    return job_data

In [88]:
def save_to_json(data, filename="incruit_jobs.json"):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"{filename}에 데이터가 저장되었습니다.")

In [89]:
def main():
    url = "https://job.incruit.com/jobdb_list/searchjob.asp?occ1=100&occ1=101&occ1=102&occ1=150&occ1=104&occ1=160&occ1=110&occ1=106&occ1=140&occ1=120&occ1=170&occ1=103&occ1=107&occ1=190&occ1=200&occ1=210&occ1=130"
    
    with requests.Session() as session:
        session.headers.update(req_header)
        res = session.get(url)

        if res.ok:
            html = res.text
            soup = BeautifulSoup(html, "html.parser")
            
            # 대분류(occ1) 정보 가져오기
            occ1_id_dict = select_occ1_id(soup)
            print(f"대분류(occ1) 개수: {len(occ1_id_dict)}")
            
            # 소분류(occ2) 정보 가져오기
            occ2_id_dict = select_occ2_id(soup, occ1_id_dict.keys())
            print(f"소분류(occ2) 개수: {len(occ2_id_dict)}")
            
            # 각 소분류별 페이지 URL 생성
            page_url_list = get_page_url_list(occ2_id_dict)
            print(f"총 페이지 수: {len(page_url_list)}")
            
            # 일부만 먼저 테스트 (전체 실행 전)
            # 전체 페이지의 10%만 먼저 테스트
            test_size = max(1, len(page_url_list) // 100)
            test_urls = page_url_list[:test_size]
            
            print(f"테스트 샘플로 {test_size}개 페이지 처리 시작...")
            job_data = classify_jobs(test_urls, occ1_id_dict, occ2_id_dict)
            
            # 테스트 결과 확인 후 전체 실행
            test_complete = input("테스트 완료. 전체 데이터를 수집하시겠습니까? (y/n): ")
            if test_complete.lower() == 'y':
                print("데이터 수집을 시작합니다...")
                job_data = classify_jobs(page_url_list, occ1_id_dict, occ2_id_dict)
                save_to_json(job_data)
            else:
                print("테스트 데이터만 저장합니다.")
                save_to_json(job_data, "incruit_test_jobs.json")
        else:
            print(f"초기 페이지 접근 오류. 에러 코드 = {res.status_code}")

In [90]:
if __name__ == "__main__":
    main()

대분류(occ1) 개수: 17
소분류(occ2) 개수: 113
총 페이지 수: 2115
테스트 샘플로 21개 페이지 처리 시작...
테스트 데이터만 저장합니다.
incruit_test_jobs.json에 데이터가 저장되었습니다.
